In [ ]:
# O primeiro passo é instalar a biblioteca cryptography
!pip3 install cryptography
# E importar todos os componentes que precisaremos utilizar
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

In [ ]:
# Aqui geramos a chave privada, ou seja, o par de números (n,d)
# O 'public_exponent' deve ser mantido como 65537 por recomendação dos criadores da biblioteca
# E o 'key_size' (tamanho da chave) pode ser qualquer número inteiro
# Números menores que 1024 são considerados inseguros. 2048 e 4096 são tamanhos padrões aceitáveis
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
# Aqui geramos a chave pública a partir da chave privada, o par de números (n,e)
public_key = private_key.public_key()

In [ ]:
# Agora vamos seguir o mesmo esquema do exemplos do slide
# Bob vai assinar uma mensagem usando sua chave privada e enviar para alice para provar que 
# ele é o dono da chave privada, e que foi ele que gerou a chave pública que Alice vai usar
message = b"Bob vai assinar essa mensagem e enviar para alice para provar que ele eh o dono da chave privada"

# Bob assina a mensagem com sua chave privada
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

In [ ]:
# Essa é a mensagem que Alice espera receber
expected_message = b"Bob vai assinar essa mensagem e enviar para alice para provar que ele eh o dono da chave privada"

# Ela vai usar a chave pública que Bob alega ser o dono para verificar que ele realmente é 
# o dono da chave privada
public_key.verify(
    signature,
    expected_message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# Por padrão o verify não retorna nada, se não ocorrer nenhum erro é porque a mensagem enviada por bob é a mesma que Alice estava esperando
# Se a mensagem não for compatível a função verify ira acusar uma exceção de InvalidSignature
# Altere o valor de expected_message e veja como esse o erro ocorre